# Bulk downloading ECCO-TCP texts #

Notebook C includes code for combining information from two different .csv files. That notebook allows us to identify which texts printed by William Bowyer are present in the ECCO-TCP corpus of TEI-encoded transcriptions and to figure out the ECCO-TCP id that corresponds to each ESTC record for a work printed by Bowyer. In this notebook, we'll actually download those texts. The code in this notebook relies on a .csv file created in notebook C, so you'll need to work through that notebook before taking on this one.

Perverse as it will seem to anybody who's ever labored over tagging a text following the TEI guidelines, we're going to throw away all of the markup to leave ourselves with a collection of plain text files. While we're at it, we'll modernize all of the long-s characters. 

## First, where are you running this notebook? ##

These notebooks were first written for use in L-100 (Digital Approaches to Bibliography and Book history) at Rare Book School in 2018. If you were a student in that course, you received an image of a virtual machine running Ubuntu Linux, along with the files that the various Jupyter Notebooks work with. If you've come to this notebook from a link at GitHub to MyBinder, you won't have those files, but I've added them to GitHub. 

NOTE: If you're running this notebook from the RBSDigitalApproaches2018 virtual machine, you'll need to be sure that you're running `jupyter notebook` with the `web-scraping` virtualenv activated. If you run into errors below related to missing libraries, quite the notebook and Jupyter Notebooks, go back to your terminalk and execute `workon web-scraping` to activate that virtualenv. (If another virtualenv is active, you may have to execute `deactivate` first.)

The first thing we need to do is to determine what file paths to use in the rest of the code below. Simply comment or uncomment the appropriate line by adding or deleting the octothorpe (#) at the beginning of the line, then run the cell.

In [ ]:
# If you're running this notebook from the RBSDigitalApproaches2018 virtual machine
# in VirtualBox, make sure the line below is uncommented (that's the default for this
# notebook.)
location = 'rbs_virtual_machine'

# If you're running this notebook at MyBinder using a link from the GitHub repository,
# comment out line 4, above, and uncomment line 8, below:
#location = 'mybinder'

if location == 'rbs_virtual_machine' :
    input_path = '/media/sf_RBSDigitalApproaches/data/0611_Tuesday_data/'
    output_path = '/media/sf_RBSDigitalApproaches/output/'
elif location == 'mybinder' :
    input_path = 'data/'
    output_path = 'output/'
    # If you're running this notebook at MyBinder.org, the code on this page will end up
    # writing files to the "output" folder. To download the files you create, go to the 
    # Jupyter Notebook "Home" tab for the repository, enter the "output" folder, click 
    # on the filename of the file you created, then click the "Download" button.

### Import libraries ###

In addition to the modules we've used before, we'll import the `os` module to create a directory in our file system.

In [ ]:
# Import libraries
import csv
import requests
from bs4 import BeautifulSoup
import re
import os

### Construct URLs ###

The .csv file we created in notebook C has the TCP ids for the texts we want to download, but we'll need to construct the full URLs for downloading.

In [ ]:
# Create a string variable with the beginning of the url we'll need to download our TCP texts from GitHub
baseurl = 'https://raw.githubusercontent.com/textcreationpartnership/'
# Create an empty list to hold the URLs we want to download
urls = []
# Open our .csv file with TCP ids, initiate the csv DictReader, and read the file a line at a time
with open(output_path + 'Bowyer_TCP_texts.csv', 'r') as infile :
    reader = csv.DictReader(infile, delimiter=',', quotechar='"')
    for row in reader :
        # Get the TCP id
        tcp_id = row['tcp_id']
        # Fill in the remainder of our URL with information based on the TCP id
        url = baseurl + tcp_id + '/master/' + tcp_id + '.xml'
        # Add our completed URL to our list of URLs
        urls.append(url)

# Let's see what we have
print(urls)

### Make our filenames for saving the files ###

We saved the URLs, which include a filename ending in .xml, but we're going to save these files as plaintext (.txt) files. Note that, because we're outside of our earlier "for" loop now, we no longer have access to the tcp_id variable (which we could have simply combined with ".txt"), so we'll have to get our filename a different way. 

(*Note*: If I were writing this as a freestanding script, rather than in a Jupyter Notebook, I'd probably do this differently, taking care of the URL construction, filename mangling, downloading, XML parsing, and file writing all inside that loop. That is, as I understand it, a relatively unsophisticated way to do things, but it certainly works...) 

In [ ]:
for url in urls :
    # First, use rpartition to split our url on the '/' character, starting from the right end and keeping 
    # the last bit. Then, take a portion of that string, beginning at character 0 and stopping four characters
    # from the end of the string (this eliminates our .xml file extension). Finally, add the new file extension
    # '.txt'.
    filename = url.rpartition('/')[-1][0:-4] + '.txt'
    print(filename)

In [ ]:
# Check to be sure that we don't already have this folder (we'd raise an error if we tried running the code in this
# cell a second time, because the directory would already exist)
if not os.path.exists(output_path + 'Bowyer_TCP/') :
    # If not, create the directory
    os.mkdir(output_path + 'Bowyer_TCP/')
for url in urls :
    filename = url.rpartition('/')[-1][:-4] + '.txt'
    # Pass the URL to the requests module and get the resource
    r = requests.get(url)
    # Pass the text that requests brings back over to teh BeautifulSoup module, using the xml parser from lxml. 
    soup = BeautifulSoup(r.text,'xml')
    # Find the "text" element of our TEI document, then get_text() to get the text content, throwing away all 
    # the markup.
    stripped = soup.find('text').get_text()
    # Use the re.sub() to find all the long-s's (we have to designate that as a unicode character with the u
    # outside the quotation marks) and replace them with short-s's.
    modernized = re.sub(u'ſ','s',stripped)
    # Open a new text file in our target directory and write our modernized text to it, encoding as utf-8
    with open(output_path + 'Bowyer_TCP/' + filename, 'wb') as file :
        file.write(modernized.encode('utf-8'))
    print('Saving ' + filename)
print("All files downloaded.")